# Capstone Project: Create a Customer Segmentation Report for Arvato Financial Services

In this project, you will analyze demographics data for customers of a mail-order sales company in Germany, comparing it against demographics information for the general population. You'll use unsupervised learning techniques to perform customer segmentation, identifying the parts of the population that best describe the core customer base of the company. Then, you'll apply what you've learned on a third dataset with demographics information for targets of a marketing campaign for the company, and use a model to predict which individuals are most likely to convert into becoming customers for the company. The data that you will use has been provided by our partners at Bertelsmann Arvato Analytics, and represents a real-life data science task.

If you completed the first term of this program, you will be familiar with the first part of this project, from the unsupervised learning project. The versions of those two datasets used in this project will include many more features and has not been pre-cleaned. You are also free to choose whatever approach you'd like to analyzing the data rather than follow pre-determined steps. In your work on this project, make sure that you carefully document your steps and decisions, since your main deliverable for this project will be a blog post reporting your findings.

In [1]:
# import libraries here; add more as necessary
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from time import time

# magic word for producing visualizations in notebook
%matplotlib inline

## Part 0: Get to Know the Data

There are four data files associated with this project:

- `Udacity_AZDIAS_052018.csv`: Demographics data for the general population of Germany; 891 211 persons (rows) x 366 features (columns).
- `Udacity_CUSTOMERS_052018.csv`: Demographics data for customers of a mail-order company; 191 652 persons (rows) x 369 features (columns).
- `Udacity_MAILOUT_052018_TRAIN.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 982 persons (rows) x 367 (columns).
- `Udacity_MAILOUT_052018_TEST.csv`: Demographics data for individuals who were targets of a marketing campaign; 42 833 persons (rows) x 366 (columns).

Each row of the demographics files represents a single person, but also includes information outside of individuals, including information about their household, building, and neighborhood. Use the information from the first two files to figure out how customers ("CUSTOMERS") are similar to or differ from the general population at large ("AZDIAS"), then use your analysis to make predictions on the other two files ("MAILOUT"), predicting which recipients are most likely to become a customer for the mail-order company.

The "CUSTOMERS" file contains three extra columns ('CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP'), which provide broad information about the customers depicted in the file. The original "MAILOUT" file included one additional column, "RESPONSE", which indicated whether or not each recipient became a customer of the company. For the "TRAIN" subset, this column has been retained, but in the "TEST" subset it has been removed; it is against that withheld column that your final predictions will be assessed in the Kaggle competition.

Otherwise, all of the remaining columns are the same between the three data files. For more information about the columns depicted in the files, you can refer to two Excel spreadsheets provided in the workspace. [One of them](./DIAS Information Levels - Attributes 2017.xlsx) is a top-level list of attributes and descriptions, organized by informational category. [The other](./DIAS Attributes - Values 2017.xlsx) is a detailed mapping of data values for each feature in alphabetical order.

In the below cell, we've provided some initial code to load in the first two datasets. Note for all of the `.csv` data files in this project that they're semicolon (`;`) delimited, so an additional argument in the [`read_csv()`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_csv.html) call has been included to read in the data properly. Also, considering the size of the datasets, it may take some time for them to load completely.

You'll notice when the data is loaded in that a warning message will immediately pop up. Before you really start digging into the modeling and analysis, you're going to need to perform some cleaning. Take some time to browse the structure of the data and look over the informational spreadsheets to understand the data values. Make some decisions on which features to keep, which features to drop, and if any revisions need to be made on data formats. It'll be a good idea to create a function with pre-processing steps, since you'll need to clean all of the datasets before you work with them.

In [ ]:
# load in the data
#azdias = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
#customers = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')
azdias = pd.read_csv('arvato_data/Udacity_AZDIAS_052018.csv', sep=';')
#customers = pd.read_csv('arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

Looks we had some trouble already while importing with some features that have mixed types values

In [ ]:
# These two featutres have mixed type float and string
azdias.iloc[:,18:20].head()

In [ ]:
azdias.CAMEO_DEUG_2015.value_counts()

In [ ]:
azdias.CAMEO_INTL_2015.value_counts()

This shows a problem with this table where there are same values that are different because of the type either int or fload

In [ ]:
# Let's fix it straightaway
def fix_cameo (df):
    """
    
    """
    df['CAMEO_DEUG_2015'] = df['CAMEO_DEUG_2015'].replace({'X': np.nan})
    df['CAMEO_DEUG_2015'] = df['CAMEO_DEUG_2015'].astype(float)
    df['CAMEO_INTL_2015'] = df['CAMEO_INTL_2015'].replace({'XX':np.nan})
    df['CAMEO_INTL_2015'] = df['CAMEO_INTL_2015'].astype(float)
    return df

azdias = fix_cameo(azdias)

So the values X and XX in these two tables are to be considered "unknown" and converted to null. So in our initial cleaning we will check the values of the documented features against the documentation ( the "DIAS Attributes - Values 2017.xlsx" file) and consider the ones that fall outside the reanges priveded as null.


In [ ]:
# Reading the features description excel file into a Dataframe
feat_info = pd.read_excel('DIAS Attributes - Values 2017.xlsx', sheet_name='Tabelle1', index_col=[0, 1, 2]).reset_index()
feat_info.drop('level_0', axis=1, inplace=True)
feat_info_levels = pd.read_excel('DIAS Information Levels - Attributes 2017.xlsx', index_col=[0, 1]).reset_index()
feat_info_levels.drop('level_0', axis=1, inplace=True) 

feat_info_levels.head()
feat_info[feat_info.Attribute=='CAMEO_DEUG_2015']

In [ ]:
# Turns out the feat_info dataset from the Excel spreadsheet has some columns that are not in the population dataset
# and vicecersa, Let's explore a bit
# These one are in the population and are described in the spreadsheet
attributes_we_have_info_about = np.intersect1d(feat_info.Attribute.unique(), np.array(azdias.columns))
# These one are in the population and are NOT described in the spreadsheet
attributes_we_dont_have_info = np.setdiff1d(np.array(azdias.columns), feat_info.Attribute.unique())
# These one are in the spreadsheet and are NOT in the population so we just ignore them for now
attributes_we_dont_care = np.setdiff1d(feat_info.Attribute.unique(), np.array(azdias.columns))

len(attributes_we_have_info_about), len(attributes_we_dont_have_info), len(attributes_we_dont_care)

Using the descriptions in the included excel spreadsheet, we will build a azdias_info dataframe where we will also add the type of the feature. Since not all features are included in the spreadsheet, we will have to add those to the dataframe, too.

In [ ]:
# We can build a dataframe with atributes and type plus description of unknown values 
unknown_values = ['unknown', 'unknown / no main age detectable', 'no transactions known', 'numeric value (typically coded from 1-10)',
                 'numeric value (typically coded from 1-3)', 'no transaction known']
azdias_info =  feat_info[(feat_info.Attribute.isin(attributes_we_have_info_about)) & (feat_info.Meaning.isin(unknown_values))]
# Add the ones who don't have unknown values

# Add columns that are in the excel spreadhseet but don't have a "unkmown value"
for feat in np.setdiff1d(feat_info.Attribute[attributes_we_have_info_about].index, azdias_info.Attribute):
    azdias_info = azdias_info.append(feat_info[feat_info.Attribute == feat].iloc[0,:2])

# Finally add columns missing if the excel spreadsheet
azdias_info = pd.concat([azdias_info, pd.DataFrame(np.setdiff1d(azdias.columns.values, azdias_info.Attribute), columns=['Attribute'])], sort=False, axis=0)

# Adding a Type column initially setting everything to 'Ordinal' 
azdias_info['Type'] = 'Ordinal'
azdias_info.to_csv('azdias_info.csv')

In [ ]:
# Reload dataframe after working on it in Excel
azdias_info = pd.read_csv('azdias_info_reload.csv', index_col=[0])

azdias_info.head()
# The Value column has values that are to be considered nulls

In [ ]:
print("Our info dataframe \"azdias_info\" has descriptions for {} features out of {}".format(azdias_info.shape[0], azdias.shape[1]))
azdias_info.Type.value_counts()
ordinal_features = azdias_info[azdias_info.Type == 'Ordinal'].Attribute.values
print("Dataset has {} ordinal features".format(len(ordinal_features)))
categorical_features = azdias_info[azdias_info.Type == 'Categorical'].Attribute.values
print("Dataset has {} categorical features".format(len(categorical_features)))
numeric_features = azdias_info[azdias_info.Type == 'Numeric'].Attribute.values
print("Dataset has {} numeric features".format(len(numeric_features)))
binary_features =   azdias_info[azdias_info.Type == 'Binary'].Attribute.values  
print("Dataset has {} binary features".format(len(binary_features)))
# Mix-Type features combine information over 2 axes and will be engineered separately
mix_type_features = azdias_info[azdias_info.Type == 'Mix-Type'].Attribute.values 
print("Dataset has {} Mix-Type features".format(len(mix_type_features)))


So we will start handling the features that are documented for now and run some cleaning scripts, then we will explore the undocumented ones and decide what to do with them.

In [ ]:
# Some utility functions
# split a string is a string otherwise return unchanged
def split_if_string(x):
    """
    
    """
    if isinstance(x, str):
        return x.split(',')
    else:
        return x
# Change ints into floats in a list and leave anything else unchanged
def to_float(x):
    """
    
    """
    try:
        x = float(x)
    except:
        pass
    return x

def is_number(s):
    """
    
    """
    try:
        float(s)
        return True
    except ValueError:
        return False

In [ ]:
# Now we will look for "undocumented" null values, that are not included in the spreadsheet

attributes_to_check = np.setdiff1d(attributes_we_have_info_about, numeric_features)

# Better align values first as it is taking as different int and floats

def check_all_values(df, feat_info):
    for attribute in attributes_to_check:
        theoretical_vals = feat_info[feat_info.Attribute == attribute].Value.apply(lambda x: split_if_string(x)).values
        theoretical_vals = np.hstack(theoretical_vals)
        theoretical_vals = [to_float(x) for x in theoretical_vals]
        actual_values = df.loc[:, attribute].value_counts().index.values
        actual_values = [to_float(x) for x in actual_values]
        #diff = set(actual_values) - set(theoretical_vals)
        diff = np.setdiff1d(actual_values, theoretical_vals)
        if diff.size > 0:
            print('Attribute {} has undocumented values {}'.format(attribute, diff))

            
check_all_values(azdias, feat_info)

So here we have:

CAMEO_DEU_2015: most common building-type within the cell => XX is a null

KBA05_MODTEMP: development of the most common car segment in the neighbourhood: we keep 6.0 although it's not documented

LP_FAMILIE_FEIN: familytyp fine: 0 is a null, must be between 1 to 11

LP_FAMILIE_GROB familytyp rough, must be 1 to 11, so 0 is a null

LP_LEBENSPHASE_FEIN: lifestage fine, must be 1 to 40, so 0 is a null

LP_LEBENSPHASE_GROB: lifestage rough, must be 1 to 12, so 0 is  a null

ORTSGR_KLS9: size of the community, 1 to 9, so 0 is a null


In [ ]:
# we can fix those above
def fix_undocumented_nulls(df):
    dxx = {'XX':np.nan}
    d0 = {0:np.nan}
    df['CAMEO_DEU_2015'] = df['CAMEO_DEU_2015'].replace(dxx)
    df['LP_FAMILIE_FEIN'] = df['LP_FAMILIE_FEIN'].replace(d0)
    df['LP_FAMILIE_GROB'] = df['LP_FAMILIE_GROB'].replace(d0)
    df['LP_LEBENSPHASE_FEIN'] = df['LP_LEBENSPHASE_FEIN'].replace(d0)
    df['LP_LEBENSPHASE_GROB'] = df['LP_LEBENSPHASE_GROB'].replace(d0)
    df['ORTSGR_KLS9'] = df['ORTSGR_KLS9'].replace(d0)
    return df

azdias =  fix_undocumented_nulls(azdias)

Now we will use the dataframe we built with the nformation of whoch values cam be considered null to replace the unlknown values with null.
We leave the features that have a value equal to zero with Meaning "no transaction known"as they represent transactional activity over a period, so we interpret is as "zero transactions".


In [ ]:
# Now let's convert the other nulls
import progressbar

def unknown2nulls(df, df_info):
    """
    
    """
    
    n_iters = df_info.shape[0]
    cnter = 0
    bar = progressbar.ProgressBar(maxval=n_iters+1, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
    bar.start()
    
    for _, info_row in df_info[df_info.Value.notnull()].iterrows():
        #print('Replacing ', info_row.Attribute)
        if isinstance(info_row.Value, str):
            d = {}
            for unknown_value in info_row.Value.split(','):
                d[float(unknown_value)] = np.nan
            df[info_row.Attribute] = df[info_row.Attribute].replace(d)
        else:
            d = {info_row.Value : np.nan}
            df[info_row.Attribute] = df[info_row.Attribute].replace(d)
        cnter+=1 
        bar.update(cnter)
    
    bar.finish()            
    return df

azdias = unknown2nulls(azdias, azdias_info)

### Choosing columns to drop
We will analyze the amount of missing data and choose which columns to drop.
Additionally some columns may be dropped for tyher reasons, for example:
- They have all unique values (useless for machine learning)
- Categorical features with high cardinality, for example more than 10-15 categories 
- when we don't have enough information to decide how to encode them
- features that are a possible source of ethical bias

In [ ]:
# Assess missing data in columns
plt.hist(azdias.isnull().sum(), bins=20)
plt.title('Distributions of null values in the Demographics')
plt.xlabel('# Null Values')
plt.ylabel('Columns');

The distribution above shows that some outlier features have more than 250,000 null values.

In [ ]:
# Investigate patterns in the amount of missing data in each column.
columns = azdias.isnull().sum().sort_values(ascending=False)
percent_missing = columns/azdias.shape[0]
data = percent_missing[percent_missing > 0.3]
plt.figure(figsize=(5,15))
sns.barplot(y=data.index, x=data.values, orient='h', color='blue')
plt.title("Attributes with more than 30% of missing values")

We will now extract a list of the columns to drop. The script will be later run on the customer data set as well, but we won't recalculate the list for the customer dataset. The idea is to use the general population dataset to "fit" our methods, like we will do later on with PCA, KMeans, etc, and then apply the fitted methods to tranform the features of the customer or other datasest. 

In [ ]:
# The columns to drop should be the same on population and customer dataset, so we leave it outside the function
columns_to_drop = [s for s, v in (azdias.isnull().sum() > azdias.shape[0] * 0.3).items() if v]
print(columns_to_drop)

In [ ]:
def drop_columns(df):
    """
    
    """
    columns_to_drop = ['AGER_TYP', 'ALTER_HH', 'ALTER_KIND1', 'ALTER_KIND2',
       'ALTER_KIND3', 'ALTER_KIND4', 'D19_BANKEN_ANZ_12',
       'D19_BANKEN_ANZ_24', 'D19_BANKEN_DATUM', 'D19_BANKEN_DIREKT',
       'D19_BANKEN_GROSS', 'D19_BANKEN_LOKAL', 'D19_BANKEN_OFFLINE_DATUM',
       'D19_BANKEN_ONLINE_DATUM', 'D19_BANKEN_REST', 'D19_BEKLEIDUNG_GEH',
       'D19_BEKLEIDUNG_REST', 'D19_BILDUNG', 'D19_BIO_OEKO',
       'D19_BUCH_CD', 'D19_DIGIT_SERV', 'D19_DROGERIEARTIKEL',
       'D19_ENERGIE', 'D19_FREIZEIT', 'D19_GARTEN', 'D19_GESAMT_ANZ_12',
       'D19_GESAMT_ANZ_24', 'D19_GESAMT_DATUM',
       'D19_GESAMT_OFFLINE_DATUM', 'D19_GESAMT_ONLINE_DATUM',
       'D19_HANDWERK', 'D19_HAUS_DEKO', 'D19_KINDERARTIKEL',
       'D19_KONSUMTYP', 'D19_KOSMETIK', 'D19_LEBENSMITTEL', 'D19_LOTTO',
       'D19_NAHRUNGSERGAENZUNG', 'D19_RATGEBER', 'D19_REISEN',
       'D19_SAMMELARTIKEL', 'D19_SCHUHE', 'D19_SONSTIGE', 'D19_SOZIALES',
       'D19_TECHNIK', 'D19_TELKO_ANZ_12', 'D19_TELKO_ANZ_24',
       'D19_TELKO_DATUM', 'D19_TELKO_MOBILE', 'D19_TELKO_OFFLINE_DATUM',
       'D19_TELKO_ONLINE_DATUM', 'D19_TELKO_REST', 'D19_TIERARTIKEL',
       'D19_VERSAND_ANZ_12', 'D19_VERSAND_ANZ_24', 'D19_VERSAND_DATUM',
       'D19_VERSAND_OFFLINE_DATUM', 'D19_VERSAND_ONLINE_DATUM',
       'D19_VERSAND_REST', 'D19_VERSI_ANZ_12', 'D19_VERSI_ANZ_24',
       'D19_VERSI_DATUM', 'D19_VERSI_OFFLINE_DATUM',
       'D19_VERSI_ONLINE_DATUM', 'D19_VERSICHERUNGEN',
       'D19_VOLLSORTIMENT', 'D19_WEIN_FEINKOST', 'EXTSEL992',
       'GEBURTSJAHR', 'KK_KUNDENTYP', 'TITEL_KZ']
    df = df.drop(columns_to_drop, axis=1)
    print("Dropped {} columns.".format(len(columns_to_drop)))
    
    return df

azdias = drop_columns(azdias)

Column with unique values (very high cardinality)

In [ ]:
# Columns with unique values, doesn't give any information
azdias.LNR.value_counts().shape

In [ ]:
# So we will drop it
#azdias.drop('LNR', axis=1, inplace=True)

Features that are a possible source of ethical bias: NATIONALITAET_KZ

In [ ]:
# NATIONALITAET_KZ: possible source of ethical bias
#azdias.drop('NATIONALITAET_KZ', axis=1, inplace=True)

In [ ]:
# ALTERSKATEGORIE_FEIN and ALTERSKATEGORIE_GROB (age through prename analysis have different number of nulls, 
# in particular ALTERSKATEGORIE_FEIN has 262947 nulls so we prefer to drop it and keep the summary information)
#azdias.drop('ALTERSKATEGORIE_FEIN', axis=1, inplace=True)

In [ ]:
# Data for this variable "type of building (residential or commercial)" is unbalanced between general population 
# and customers: The gen pop has only one commercial building (??) and customers has no residential building
# So we choose to frop it
azdias.GEBAEUDETYP.value_counts(), customers.GEBAEUDETYP.value_counts()

In [ ]:
# We put all together in a function

def drop_other_columns(df):
    """
    
    """
    other_columns_to_drop = ['LNR', 'NATIONALITAET_KZ', 'ALTERSKATEGORIE_FEIN', 'GEBAEUDETYP']
    df = df.drop(other_columns_to_drop, axis=1)
    return df

azdias = drop_other_columns(azdias)

Exploring Categiorical Features with Multi-values, look for high cardnality features.

In [ ]:
# Let's have a look at the other categorical features (the ones who are left)

categorical_features = np.intersect1d(categorical_features, azdias.columns.values)

categorical_to_drop = []
for categ in categorical_features:
    num_categs = azdias[categ].value_counts().shape[0]
    if num_categs > 15:
        print("Feature {} has {} categories".format(categ, num_categs))
        categorical_to_drop.append(categ)
   

We will drop CAMEO_DEU_2015 (redundant), EINGEFUEGT_AM (has thousands of dates, not interesting), D19_LETZTER_KAUF_BRANCHE, VERDICHTUNGSRAUM, while it looks  CAMEO_DEUG_2015: CAMEO classification 2015 - Uppergroup carris important information we want to keep

In [ ]:
def drop_categorical(df):
    """
    """
    categorical_to_drop = ['CAMEO_DEU_2015','D19_LETZTER_KAUF_BRANCHE','EINGEFUEGT_AM','VERDICHTUNGSRAUM']
    df = df.drop(categorical_to_drop, axis=1)
    return df


azdias = drop_categorical(azdias)

### Missing Data in rows

In [ ]:
# How much data is missing in each row of the dataset?
plt.hist(azdias.isnull().sum(axis=1), bins=40)
plt.title('Distributions of null values in the rows Demographics')
plt.xlabel('# Null Values')
plt.ylabel('Rows')

In [ ]:
# We can see that the large majority of the population has less than 50 null fatures.
# we can remove the datapoints with more than 50 nulls in each row
def drop_rows(df, nulls=50):
    """
    
    """
    df = df.dropna(thresh=df.shape[1]-nulls)
    df = df.reset_index()
    del df['index']
    return df

azdias = drop_rows(azdias)

In [ ]:
# Now we can check again....
plt.hist(azdias.isnull().sum(axis=1), bins=40)
plt.title('Distributions of null values in the rows Demographics')
plt.xlabel('# Null Values')
plt.ylabel('Rows')

In [ ]:
# Let's see how many rows, columns we are left with
azdias.shape

### Features Re-encoding
Some categorical features need re-encoding. We will look at the spreadsheet to choose which ones to re-encode.
Since the unsupervised learning techniques to be used will only work on data that is encoded numerically, we need to make a few encoding changes or additional assumptions to be able to make progress. In addition, while almost all of the values in the dataset are encoded using numbers, not all of them represent numeric values.

- For numeric and interval data, these features can be kept without changes.
- Most of the variables in the dataset are ordinal in nature. While ordinal values may technically be non-linear in spacing, we make the simplifying assumption that the ordinal variables can be treated as being interval in nature (that is, kept without any changes).
- Special handling will be necessary for the remaining two variable types: categorical, and 'mixed'.

Are there binary categorical features that need re-encoding?
Are there multi-level categorical features that need re-encoding?


In [ ]:
# Binary categorical features
# OST_WEST_KZ: flag indicating the former GDR/FRG
#azdias['OST_WEST_KZ'].value_counts()
def fix_binary_features(df):
    """
    """
    ost_west_dict = {'OST_WEST_KZ': {'W':0, 'O':1}}
    df = df.replace(ost_west_dict)
    return(df)

azdias = fix_binary_features(azdias)

OBSOLETE: Multi-level categoricals (three or more values) will be one-hot encoded using multiple dummy variables.

In [ ]:
# Multi value features

categorical_features = np.intersect1d(azdias_info[azdias_info.Type=='Categorical'].Attribute.values, azdias.columns.values)
azdias_info[azdias_info.Attribute.isin(categorical_features)]

In [ ]:
azdias.LP_FAMILIE_GROB.value_counts()

In [ ]:
# OBSOLETE: Actually one-hot encoding doesn't work well with Tree-based methods liked Random Forest or Gradient Boost, 
# so we remove it for now
# We drop LP_FAMILIE_GROB and one-hot encode the other categorical feats

one_hot_encode_cols = np.intersect1d(azdias_info[azdias_info.Type=='Categorical'].Attribute.values, azdias.columns.values)
one_hot_encode_cols = np.setdiff1d(one_hot_encode_cols, ['LP_FAMILIE_GROB'])

print(one_hot_encode_cols)

In [ ]:
def encode_categorical(df, azdias_info):
    """
    """
    
    one_hot_encode_cols = ['CAMEO_DEUG_2015', 'CJT_GESAMTTYP', 'FINANZTYP',
       'GEBAEUDETYP_RASTER', 'GEMEINDETYP', 'GFK_URLAUBERTYP',
       'HEALTH_TYP', 'LP_FAMILIE_FEIN', 'LP_LEBENSPHASE_FEIN',
       'LP_LEBENSPHASE_GROB', 'REGIOTYP', 'RETOURTYP_BK_S', 'SHOPPER_TYP',
       'VK_DHT4A', 'WOHNLAGE', 'ZABEOTYP']
    
    #df = df.drop('LP_FAMILIE_GROB', axis=1)
    #df = pd.get_dummies(df, columns=one_hot_encode_cols)
    
    return df

#azdias = encode_categorical(azdias, azdias_info)

In [ ]:
azdias.shape

### Engineer Mixed-Type Features
There are two of features that we marked as "mixed" in the azdias_info dataframe that require special treatment in order to be included in the analysis. 
- "PRAEGENDE_JUGENDJAHRE" combines information on three dimensions: generation by decade, movement (mainstream vs. avantgarde), and nation (east vs. west). While there aren't enough levels to disentangle east from west, you should create two new variables to capture the other two dimensions: an interval-type variable for decade, and a binary variable for movement.
- "CAMEO_INTL_2015" combines information on two axes: wealth and life stage. Break up the two-digit codes by their 'tens'-place and 'ones'-place digits into two new ordinal variables (which, for the purposes of this project, is equivalent to just treating them as their raw numeric values).
- WOHNLAGE (Residential Area): combines neighboorhood quality and rural/non-rural information. We create two new features NEIGHBORHOOD_QUALITY (Ordinal, vaulues from 1 to 5) and RURAL (Binary, 0= non-rural, 1=rural)
- LP_LEBENSPHASE_FEIN has information over 3 dimensions: Income (from low-income to top-income), Age (younger/middle/older/advanced/retirement) and Family Status (single/couples/families/multiperson households). However his contents are summarized in the LP_LEBENSPHASE_GROB feature that is much easier to interpret, with values from 1 (single low-income) to 12 (high-income earners of higher age from multiperson households). So we keep this last one and we will drop LP_LEBENSPHASE_FEIN


In [ ]:
azdias_info[azdias_info.Type == 'Mix-Type']

In [ ]:
azdias.LP_LEBENSPHASE_FEIN.isnull().sum(), azdias.LP_LEBENSPHASE_GROB.isnull().sum()
azdias.LP_LEBENSPHASE_GROB.value_counts()

In [ ]:
rural_dict = {1.0:0, 2.0:0, 3.0:0, 4.0:0, 5.0:0, 7.0:1, 8.0:1}
#azdias.WOHNLAGE.value_counts()
azdias['NEIGHBORHOOD_QUALITY'] = np.nan
azdias['NEIGHBORHOOD_QUALITY']  = azdias[(azdias['WOHNLAGE'] > 0) & (azdias['WOHNLAGE'] < 7)]['WOHNLAGE']
azdias['NEIGHBORHOOD_QUALITY'].value_counts()
azdias['RURAL'] = np.nan
azdias['RURAL'] = azdias['WOHNLAGE'].map(rural_dict)
azdias.RURAL.value_counts()

In [ ]:
# Investigate "PRAEGENDE_JUGENDJAHRE" and engineer two new variables.
azdias['PRAEGENDE_JUGENDJAHRE'].value_counts()


WOHNLAGE
Residential Area
- -1	unknown
-	0	no score calculated
-	1	very good neighbourhood
-	2	good neighbourhood
-	3	average neighbourhood
-	4	poor neighbourhood
-	5	very poor neighbourhood
-	7	rural neighbourhood
-	8	new building in rural neighbourhood






PRAEGENDE_JUGENDJAHRE
Dominating movement of person's youth (avantgarde vs. mainstream; east vs. west)
- -1: unknown
-  0: unknown
-  1: 40s - war years (Mainstream, E+W)
-  2: 40s - reconstruction years (Avantgarde, E+W)
-  3: 50s - economic miracle (Mainstream, E+W)
-  4: 50s - milk bar / Individualisation (Avantgarde, E+W)
-  5: 60s - economic miracle (Mainstream, E+W)
-  6: 60s - generation 68 / student protestors (Avantgarde, W)
-  7: 60s - opponents to the building of the Wall (Avantgarde, E)
-  8: 70s - family orientation (Mainstream, E+W)
-  9: 70s - peace movement (Avantgarde, E+W)
- 10: 80s - Generation Golf (Mainstream, W)
- 11: 80s - ecological awareness (Avantgarde, W)
- 12: 80s - FDJ / communist party youth organisation (Mainstream, E)
- 13: 80s - Swords into ploughshares (Avantgarde, E)
- 14: 90s - digital media kids (Mainstream, E+W)
- 15: 90s - ecological awareness (Avantgarde, E+W)


German CAMEO: Wealth / Life Stage Typology, mapped to international code
- -1: unknown
- 11: Wealthy Households - Pre-Family Couples & Singles
- 12: Wealthy Households - Young Couples With Children
- 13: Wealthy Households - Families With School Age Children
- 14: Wealthy Households - Older Families &  Mature Couples
- 15: Wealthy Households - Elders In Retirement
- 21: Prosperous Households - Pre-Family Couples & Singles
- 22: Prosperous Households - Young Couples With Children
- 23: Prosperous Households - Families With School Age Children
- 24: Prosperous Households - Older Families & Mature Couples
- 25: Prosperous Households - Elders In Retirement
- 31: Comfortable Households - Pre-Family Couples & Singles
- 32: Comfortable Households - Young Couples With Children
- 33: Comfortable Households - Families With School Age Children
- 34: Comfortable Households - Older Families & Mature Couples
- 35: Comfortable Households - Elders In Retirement
- 41: Less Affluent Households - Pre-Family Couples & Singles
- 42: Less Affluent Households - Young Couples With Children
- 43: Less Affluent Households - Families With School Age Children
- 44: Less Affluent Households - Older Families & Mature Couples
- 45: Less Affluent Households - Elders In Retirement
- 51: Poorer Households - Pre-Family Couples & Singles
- 52: Poorer Households - Young Couples With Children
- 53: Poorer Households - Families With School Age Children
- 54: Poorer Households - Older Families & Mature Couples
- 55: Poorer Households - Elders In Retirement
- XX: unknown

In [ ]:
# Re-engineer variables
def engineer_variables(df):
    """
    """
    # MAINSTREAM AND GENERATION
    mainstream_dict = {1.0:1, 2.0:2, 3.0:1, 4.0:2, 5.0:1, 6.0:2, 7.0:2, 8.0:1, 9.0:2, 10.0:1, 11.0:2, 12.0:1, 13.0:2, 14.0:1, 15.0:2}
    df['MAINSTREAM'] = np.nan
    df['MAINSTREAM'] = df['PRAEGENDE_JUGENDJAHRE'].map(mainstream_dict)
    generation_dict = {1.0:1, 2.0:1, 3.0:2, 4.0:2, 5.0:3, 6.0:3, 7.0:3, 8.0:4, 9.0:4, 10.0:5, 11.0:5, 12.0:5, 13.0:5, 14.0:6, 15.0:6}
    df['GENERATION'] = np.nan
    df['GENERATION'] = df['PRAEGENDE_JUGENDJAHRE'].map(generation_dict)  
    df['WEALTH'] = df['CAMEO_INTL_2015'].apply(lambda x: np.floor_divide(float(x), 10) if float(x) else np.nan)
    df['FAMILY'] = df['CAMEO_INTL_2015'].apply(lambda x: np.mod(float(x), 10) if float(x) else np.nan) 
    
    rural_dict = {1.0:0, 2.0:0, 3.0:0, 4.0:0, 5.0:0, 7.0:1, 8.0:1}
    df['NEIGHBORHOOD_QUALITY'] = np.nan
    df['NEIGHBORHOOD_QUALITY']  = df[(df['WOHNLAGE'] > 0) & (df['WOHNLAGE'] < 7)]['WOHNLAGE']
    df['NEIGHBORHOOD_QUALITY'].value_counts()
    df['RURAL'] = np.nan
    df['RURAL'] = df['WOHNLAGE'].map(rural_dict)

    # Remove old variables
    df = df.drop(['PRAEGENDE_JUGENDJAHRE', 'CAMEO_INTL_2015', 'WOHNLAGE', 'LP_LEBENSPHASE_FEIN'],axis=1)
    return df

azdias = engineer_variables(azdias)   

We have decided not to one-hot encode categorical variables as it does not help when using Tree-based methods like Decisopn Trees, RandomForest or GradientBoost

In [ ]:
# Now we will put all the cleaning into one function, so that we can apply it to the customer database later on
def clean_data(df, azdias_info, cross_validation=False):
    """
    """
    print("Converting unknown to nulls...")
    df = fix_cameo(df)
    df = fix_undocumented_nulls(df)
    df = unknown2nulls(df, azdias_info)
    print("Dropping columns...")
    df = drop_columns(df)
    df = drop_other_columns(df)
    df = drop_categorical(df)
    if cross_validation == False:
        print("Dropping rows...")
        df = drop_rows(df)
    df = fix_binary_features(df)
    # We don't one-hot encode as it doesn't help with Tree based methods we intend to use later on
    #print("One hot encoding...")
    #df = encode_categorical(df, azdias_info)
    print("Engineering new variables...")
    df = engineer_variables(df)
    print("Done.")

    return df

#azdias = clean_data(azdias, azdias_info)

In [ ]:
azdias.to_feather('azdias_cleaned.feather')

In [ ]:
# Without one-hot encoding
cat_cols = np.intersect1d(azdias_info[azdias_info.Type=='Categorical'].Attribute.values, azdias.columns.values)
num_cols = np.intersect1d(azdias_info[azdias_info.Type=='Numeric'].Attribute.values, azdias.columns.values)
ord_cols = np.intersect1d(azdias_info[azdias_info.Type=='Ordinal'].Attribute.values, azdias.columns.values)
mix_cols = np.intersect1d(azdias_info[azdias_info.Type=='Mix-Type'].Attribute.values, azdias.columns.values)
bin_cols = np.intersect1d(azdias_info[azdias_info.Type=='Binary'].Attribute.values, azdias.columns.values)
bin_cols = np.append(bin_cols, 'MAINSTREAM')
ord_cols = np.concatenate((ord_cols, ['GENERATION', 'WEALTH', 'FAMILY', 'NEIGHBORHOOD_QUALITY', 'RURAL']))
print(len(cat_cols))
print(len(num_cols))
print(len(ord_cols))
print(len(mix_cols))
print(len(bin_cols))
azdias.shape, len(cat_cols)+len(num_cols)+len(ord_cols)+len(mix_cols)+len(bin_cols)

In [ ]:
azdias.shape

## Feature Transformation

### Apply Feature Scaling

Before we apply dimensionality reduction techniques to the data, we need to perform feature scaling so that the principal component vectors are not influenced by the natural differences in scale for features.  In this substep, we will  check the following:

- Before applying the scaler to the data, we need to make sure that we've cleaned the DataFrame of the remaining missing values. This could be as simple as just removing all data points with missing data, thereby losing a significative percentage of the data, or applying an [Imputer](http://scikit-learn.org/stable/modules/generated/sklearn.preprocessing.Imputer.html) to replace all missing values. 
- We will separate categorical variables from ordinal and numeric ones, as we will use two different strategies for imputing. Then we intersect with the columns that have null values and finally we add the new engineered Mixed-Type Features
- When imputing, the biggest concern is introducing bias into the data, therefore it's advisable to use different strategies depending on the type of the data. We will use most frequent values for binary variable, median for ordinal ones, and categorical variables have been already handled by creating dummy columns. Finally for numeric columns, if they are skewed we will apply a logarithmic tranform, then impute the median.



In [ ]:
azdias = pd.read_feather('azdias_cleaned.feather', nthreads=2)

In [ ]:
azdias.shape
#azdias.columns[azdias.isnull().any()]

In [ ]:
d19_cols = [col for col in azdias if col.startswith('D19')]
for d19 in d19_cols:
    print("Column {} has {} nulls".format(d19, azdias[d19].isnull().sum()))

d19_features = ['D19_BANKEN_ONLINE_QUOTE_12', 'D19_GESAMT_ONLINE_QUOTE_12', 'D19_TELKO_ONLINE_QUOTE_12', 'D19_VERSAND_ONLINE_QUOTE_12', 'D19_VERSI_ONLINE_QUOTE_12']

In [ ]:
azdias.columns[azdias.isnull().any()]

In [ ]:
# OBSOLETE

# Now for the other columns we need to know which are numerical, ordinal, categorical. 
#And we shoudn't forget the last 4 column we added
columns_with_nulls = azdias.columns[azdias.isnull().any()]
# The columns we added manually are the last 4, plus KBA05_GBZ' that need to be added to the list
# because it has nulls in the customer dataset, otherwise the pca transform will fail

cols_to_impute = np.setdiff1d(columns_with_nulls, d19_impute_zero)

numeric2impute = azdias_info[(azdias_info.Attribute.isin(cols_to_impute)) & (azdias_info.Type =='Numeric')].Attribute.values
categorical2impute = azdias_info[(azdias_info.Attribute.isin(cols_to_impute)) & (azdias_info.Type =='Categorical')].Attribute.values
ordinal2impute = azdias_info[(azdias_info.Attribute.isin(cols_to_impute)) & (azdias_info.Type =='Ordinal')].Attribute.values
binary2impute = azdias_info[(azdias_info.Attribute.isin(cols_to_impute)) & (azdias_info.Type =='Binary')].Attribute.values
binary2impute = np.append(binary2impute, 'MAINSTREAM')
ordinal2impute = np.concatenate((ordinal2impute, ['GENERATION', 'WEALTH', 'FAMILY', 'KBA05_GBZ']))
categorical2impute

In [ ]:
azdias.LP_LEBENSPHASE_GROB.isnull().sum()

### Transform Pipeline
We have a problem with the D19_ that have a very high number of null values. 
Imputing with a median would introduce an unacceptable bias as the second most frequent value is usually in the few thousands.
Therefor we will:
- impute a zero in the D19_.._QUOTE12 variables (equivalent to "no Online-transactions within the last 12 months")

In [ ]:
d19_cols = [col for col in azdias if col.startswith('D19')]
for d19 in d19_cols:
    print("Column {} has {} nulls".format(d19, azdias[d19].isnull().sum()))

d19_features = ['D19_BANKEN_ONLINE_QUOTE_12', 'D19_GESAMT_ONLINE_QUOTE_12', 'D19_TELKO_ONLINE_QUOTE_12', 'D19_VERSAND_ONLINE_QUOTE_12', 'D19_VERSI_ONLINE_QUOTE_12']




In [ ]:
# Alternative approach: we fit the imputer to all columns instead of only columns with nulls, so to have more flexibility 
# when applying the imputer to the customer or other datasets

ordinal_features = np.intersect1d(azdias_info[azdias_info.Type == 'Ordinal'].Attribute.values, azdias.columns.values)
ordinal_features = np.setdiff1d(ordinal_features, d19_features)
# This one should be empty because we one-hot-encoded them
categorical_features = np.intersect1d(azdias_info[azdias_info.Type == 'Categorical'].Attribute.values, azdias.columns.values)
numeric_features = np.intersect1d(azdias_info[azdias_info.Type == 'Numeric'].Attribute.values, azdias.columns.values)
binary_features =   np.intersect1d(azdias_info[azdias_info.Type == 'Binary'].Attribute.values, azdias.columns.values)
# Mix-Type
mix_type_features = np.intersect1d(azdias_info[azdias_info.Type == 'Mix-Type'].Attribute.values, azdias.columns.values)

# Add Re-engineerd variables
binary_features = np.append(binary_features, 'MAINSTREAM')
ordinal_features = np.concatenate((ordinal_features, ['GENERATION', 'WEALTH', 'FAMILY',  'NEIGHBORHOOD_QUALITY', 'RURAL']))

In [ ]:
azdias[list(numeric_features)+list(binary_features)+list(ordinal_features)].head()

In [ ]:
from sklearn.preprocessing import FunctionTransformer, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer

def log_adjusted(x):
    return np.log1p(1+x)

def inv_log_adjusted(x):
    return np.expm1(x)-1
   

log_transform = ('log_transform', FunctionTransformer(func = log_adjusted, inverse_func=inv_log_adjusted, validate=False))
log_impute = ('log_impute', SimpleImputer(missing_values=np.nan, strategy='median'))
log_scale = ('log_scale', StandardScaler())

log_pipeline = Pipeline([log_transform, log_impute, log_scale])

binary_pipeline = Pipeline([('binary_impute', SimpleImputer(missing_values=np.nan, strategy='most_frequent'))])

#cat_pipeline = Pipeline([('cat_dummy', etl.DummiesTransformer())])
cat_pipeline = Pipeline([('cat_impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
                          ('cat_scale', StandardScaler())])
ordinal_impute = ('ordinal_impute', SimpleImputer(missing_values=np.nan, strategy='median'))
ordinal_scale = ('ordinal_scale', StandardScaler())

ordinal_pipeline = Pipeline([ordinal_impute, ordinal_scale])

d19_pipeline = Pipeline([('d19_impute', SimpleImputer(missing_values=np.nan, strategy='constant', fill_value=0)),
                          ('d19_scale', StandardScaler())])

mix_pipeline = Pipeline([('mix_impute', SimpleImputer(missing_values=np.nan, strategy='median')),
                          ('mix_scale', StandardScaler())])

transformers = [('log', log_pipeline, numeric_features),
                ('binary', binary_pipeline, binary_features),
                ('cat', cat_pipeline, categorical_features),
                ('ordinal', ordinal_pipeline, ordinal_features),
                ('mix', mix_pipeline, mix_type_features),
                ('d19', d19_pipeline, d19_features)]

c_transformer = ColumnTransformer(transformers=transformers)



#columns_ex_cat = list(numeric_features)+list(binary_features)+list(ordinal_features) 
azdias_transformed = c_transformer.fit_transform(azdias)

azdias_transformed.shape, azdias.shape

In [ ]:
#Now we can rebuild the DataFrame if we need
azdias_columns = list(numeric_features)+list(binary_features)+list(categorical_features)+list(ordinal_features)+list(mix_type_features)+list(d19_features)
azdias_transformed = pd.DataFrame(azdias_transformed, columns=azdias_columns)

In [ ]:
#We now have a look at the distributions of numerical variables
plt.figure(figsize=(20,5))
plt.subplot(2, 3, 1)
plt.hist(azdias.ANZ_HAUSHALTE_AKTIV.dropna());
plt.subplot(2, 3, 2)
plt.hist(azdias.ANZ_HH_TITEL.dropna());
plt.subplot(2, 3, 3)
plt.hist(azdias.ANZ_PERSONEN.dropna());
plt.subplot(2, 3, 4)
plt.hist(azdias.ANZ_KINDER.dropna());
plt.subplot(2, 3, 5)
plt.hist(azdias.ANZ_TITEL.dropna());
plt.subplot(2, 3, 6)
plt.hist(azdias.ANZ_STATISTISCHE_HAUSHALTE.dropna());
plt.show()

The therendistributiones look all right skewed: when data are very non-normal (and e.g. needs a log-transformation to give approximate normality), then any imputation method assuming normality may not perform so well. Therefore we will log-tranform the numerical features before applying imputation

In [ ]:
# Log transform numeric features
#from sklearn.preprocessing import FunctionTransformer
def log_transform_numeric(df):
    """
    
    """
    df['ANZ_HAUSHALTE_AKTIV'] = df['ANZ_HAUSHALTE_AKTIV'].apply(lambda x: np.log1p(1+x) if x != np.nan else x)
    df['ANZ_HH_TITEL'] = df['ANZ_HH_TITEL'].apply(lambda x: np.log1p(1+x) if x != np.nan else x)
    df['ANZ_PERSONEN'] = df['ANZ_PERSONEN'].apply(lambda x: np.log1p(1+x) if x != np.nan else x)
    df['ANZ_TITEL'] = df['ANZ_TITEL'].apply(lambda x: np.log1p(1+x) if x != np.nan else x)
    df['ANZ_KINDER'] = df['ANZ_KINDER'].apply(lambda x: np.log1p(1+x) if x != np.nan else x)
    df['KBA13_ANZAHL_PKW'] = df['KBA13_ANZAHL_PKW'].apply(lambda x: np.log1p(1+x) if x != np.nan else x)
    df['ANZ_STATISTISCHE_HAUSHALTE'] = df['ANZ_STATISTISCHE_HAUSHALTE'].apply(lambda x: np.log1p(1+x) if x != np.nan else x)
    
    return df

azdias = log_transform_numeric(azdias)


In [ ]:
#We can have a look at the distributions of numerical variables again
plt.figure(figsize=(20,5))
plt.subplot(2, 3, 1)
plt.hist(azdias.ANZ_HAUSHALTE_AKTIV.dropna());
plt.subplot(2, 3, 2)
plt.hist(azdias.ANZ_HH_TITEL.dropna());
plt.subplot(2, 3, 3)
plt.hist(azdias.ANZ_PERSONEN.dropna());
plt.subplot(2, 3, 4)
plt.hist(azdias.ANZ_KINDER.dropna());
plt.subplot(2, 3, 5)
plt.hist(azdias.ANZ_TITEL.dropna());
plt.subplot(2, 3, 6)
plt.hist(azdias.ANZ_STATISTISCHE_HAUSHALTE.dropna());
plt.show()

A bit better, but still skewed. So we will use meadian to impute instead of mean.

In [ ]:
#OBSOLETE
#Doesn't work well like this: we need a fit and a transform method, like in a pipeline
from sklearn.impute import SimpleImputer
def impute_fit_transform(df):
    """
    
    """
    
    imputer_ordinal = SimpleImputer(strategy='median')
    df[ordinal_features] = imputer_ordinal.fit_transform(df[ordinal_features])
    # Numeric features
    imputer_numeric = SimpleImputer(strategy='median')
    df[numeric_features] = imputer_numeric.fit_transform(df[numeric_features])
    # Binary features
    imputer_binary = SimpleImputer(strategy='most_frequent')
    df[binary_features] = imputer_binary.fit_transform(df[binary_features])  
    
    # Set aside the fitted objects
    imputers = [imputer_ordinal, imputer_numeric, imputer_binary]
        
    # Let's not forget the d19
    df[d19_impute_zero] = df[d19_impute_zero].fillna(0)

    return df, imputers


def impute_transform(df, imputers):
    """
    
    """
    # Extract imputers and transform only
    imputer_ordinal = imputers[0]
    imputer_numeric = imputers[1]
    imputer_binary = imputers[2]
    

    df[ordinal_features] = imputer_ordinal.transform(df[ordinal_features])
    # Numeric features

    df[numeric_features] = imputer_numeric.transform(df[numeric_features])
    # Binary features

    df[binary_features] = imputer_binary.transform(df[binary_features])  
        
    # Let's not forget the d19
    df[d19_impute_zero] = df[d19_impute_zero].fillna(0)

    return df

azdias, imputers = impute_fit_transform(azdias)

In [ ]:
# OBSOLETE
# Now we can impute: we need to save the fitted objects to use them with the customer data
from sklearn.impute import SimpleImputer
def impute_nulls(df, azdias_info, imputers = []):
    """
    
    """
    #The columns to impute should not be recomputed for the customer dataset again, so we leave it outside the function
    
    if imputers == []:
        # Ordinal features
        imputer_ordinal = SimpleImputer(strategy='median')
        df[ordinal2impute] = imputer_ordinal.fit_transform(df[ordinal2impute])
        # Numeric features
        imputer_numeric = SimpleImputer(strategy='median')
        df[numeric2impute] = imputer_numeric.fit_transform(df[numeric2impute])
        # Binary features
        imputer_binary = SimpleImputer(strategy='most_frequent')
        df[binary2impute] = imputer_binary.fit_transform(df[binary2impute])  
        # Set aside the fitted objects
        imputers = [imputer_ordinal, imputer_numeric, imputer_binary]
    else:
        # Extract imputers and transform only
        imputer_ordinal = imputers[0]
        imputer_numeric = imputers[1]
        imputer_binary = imputers[2]
        
        df[ordinal2impute] = imputer_ordinal.transform(df[ordinal2impute])
        df[numeric2impute] = imputer_numeric.transform(df[numeric2impute])
        df[binary2impute] = imputer_binary.transform(df[binary2impute])
    
    # Let's not forget the d19
    df[d19_impute_zero] = df[d19_impute_zero].fillna(0)
        
    return df, imputers

azdias, imputers = impute_nulls(azdias, azdias_info)

In [ ]:
# Now we can apply Standard Scaling to all numerical and ordinal columns
from sklearn.preprocessing import StandardScaler

#ordinal_features = np.intersect1d(azdias_info[azdias_info.Type=='Ordinal'].Attribute.values, azdias.columns.values)
#ordinal_features = np.concatenate((ordinal_features, ['GENERATION', 'WEALTH', 'FAMILY']))
#numeric_features = np.intersect1d(azdias_info[azdias_info.Type=='Numeric'].Attribute.values, azdias.columns.values)
features_to_scale = np.concatenate((ordinal_features, numeric_features))

def scale_features(df, scaler=None):
    """
    
    """
    if scaler == None:
        scaler=StandardScaler()
        df[features_to_scale] = scaler.fit_transform(df[features_to_scale])
    else:
        # Transform only
        df[features_to_scale] = scaler.transform(df[features_to_scale])
    return df, scaler

azdias, scaler = scale_features(azdias)

In [ ]:
# Can be put all together in a function
def feature_transformation(df, azdias_info, imputers = [], scaler=None):
    """
    
    """
    print("Log Transforming numeric features...")
    df = log_transform_numeric(df)
    print("Imputing missing values...")
    #if imputers == []:
    #    df, imputers = impute_fit_transform(df)
    #else:
    #    df = impute_transform(df, imputers)
    df, imputers = impute_nulls(df, azdias_info, imputers)    
    print("Scaling features...")
    df, scaler = scale_features(df, scaler)
    print("Done.")
    return df, imputers, scaler

#azdias, imputers, scaler = feature_transformation(azdias, azdias_info)

In [ ]:
imputers, scaler

In [ ]:
azdias.to_feather('azdias_transformed.feather')
#azdias.shape

## Part 1: Customer Segmentation Report

The main bulk of your analysis will come in this part of the project. Here, you should use unsupervised learning techniques to describe the relationship between the demographics of the company's existing customers and the general population of Germany. By the end of this part, you should be able to describe parts of the general population that are more likely to be part of the mail-order company's main customer base, and which parts of the general population are less so.

### PCA
On the scaled data, we are now ready to apply dimensionality reduction techniques.

- Use sklearn's [PCA](http://scikit-learn.org/stable/modules/generated/sklearn.decomposition.PCA.html) class to apply principal component analysis on the data, thus finding the vectors of maximal variance in the data. To start, you should not set any parameters (so all components are computed) or set a number of components that is at least half the number of features (so there's enough features to see the general trend in variability).
- Check out the ratio of variance explained by each principal component as well as the cumulative variance explained. Try plotting the cumulative or sequential values using matplotlib's [`plot()`](https://matplotlib.org/api/_as_gen/matplotlib.pyplot.plot.html) function. Based on what you find, select a value for the number of transformed features you'll retain for the clustering part of the project.
- Once you've made a choice for the number of components to keep, make sure you re-fit a PCA instance to perform the decided-on transformation.

In [ ]:
#azdias = pd.read_feather('azdias_scaled.feather', nthreads=2)
from sklearn.decomposition import PCA

In [ ]:
pca = PCA()
pca.fit(azdias_transformed)
azdias_pca = pca.transform(azdias_transformed)

In [ ]:
def show_pca(pca):
    cumulative_ratios=np.zeros(len(pca.explained_variance_ratio_))
    for i in range(len(pca.explained_variance_ratio_)):
        cumulative_ratios[i]=np.sum(pca.explained_variance_ratio_[:i])
    plt.figure(figsize=(20,10))
    plt.plot(pca.explained_variance_ratio_)
    plt.plot(cumulative_ratios)
    plt.xlabel("Components")
    plt.ylabel("Explained Variance Ratio %")
    plt.title("PCA Components Explained Variance Ratios")
    plt.yticks(np.arange(0, 1, step=0.05))
    plt.xticks(np.arange(0, len(pca.explained_variance_ratio_)+2, step= (len(pca.explained_variance_ratio_) // 20)))
    plt.grid(linewidth=0.1)
    plt.legend(['Variance Ratio', 'Cumulative'], loc='center right')
    
#show_pca(pca)

In [ ]:
np.sum(pca.explained_variance_ratio_[:140])

In [ ]:
azdias_transformed.shape

The pca without parameters returned all the 289 components (same as n. of features)
The plot shows that over 80% of the variance is explained by the first 90 components, 90% by the first 140 components and after that it increases very slowly

So we will run it again but this time we want only 140 components. Also we are saving the pca object so we can ru it 
on the customer dataset later on

In [ ]:
def apply_pca(df, n_components=300):
    """
    
    """
    pca = PCA(n_components=n_components)
    pca.fit(df)
    df_pca = pca.transform(df)
    
    return df_pca, pca

azdias_pca, pca = apply_pca(azdias_transformed, 140)

In [ ]:
show_pca(pca)

In [ ]:
import pickle
azdias.to_feather('azdias_pca.feather')

# Save to file in the current working directory
pkl_filename = "pca.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(pca, file)

In [ ]:
def pca_results(full_dataset, pca, component):
    '''
    Create a DataFrame of the PCA results
    Includes dimension feature weights and explained variance
    Visualizes the PCA results
    '''

    # Dimension indexing
    dimensions = ['Dimension {}'.format(i) for i in range(1,len(pca.components_)+1)]

    # PCA components
    components = pd.DataFrame(np.round(pca.components_, 4), columns = full_dataset.keys())
    components.index = dimensions

    # PCA explained variance
    ratios = pca.explained_variance_ratio_.reshape(len(pca.components_), 1)
    variance_ratios = pd.DataFrame(np.round(ratios, 4), columns = ['Explained Variance'])
    variance_ratios.index = dimensions

    # Create a bar plot visualization
    fig, ax = plt.subplots(figsize = (20,10))

    # Plot the feature weights as a function of the components
    features_to_show = components.iloc[component - 1].sort_values(ascending=False)
    features_to_show = features_to_show[np.absolute(features_to_show.values) >= 0.01]
    #components.iloc[component - 1].sort_values(ascending=False).plot(ax = ax, kind = 'bar');
    features_to_show.plot(ax = ax, kind = 'bar');
    ax.set_ylabel("Feature Weights")
    ax.set_xticklabels(list(features_to_show.keys()), rotation=90)
    ax.set_xlabel("Features")

    # Display the explained variance ratios
    #for i, ev in enumerate(pca.explained_variance_ratio_):
    ev = pca.explained_variance_ratio_[component-1]
    
    plt.title("Component {} Explained Variance: {:.2f}%".format(component, ev*100))

    # Return a concatenated DataFrame
    #return pd.concat([variance_ratios, components], axis = 1).iloc[component - 1].sort_values(ascending=False)
    return features_to_show

### First Component

In [ ]:
pca_results(azdias_transformed, pca, 1)

Positive correlations
PLZ8_ANTG3: # of 6-10 family houses in the area
PLZ8_ANTG4 # of +10 families in the area  
KBA13_BAUMAX : Building type
ANZ_HAUSHALTE_AKTIV # of households in the building
WEALTH: here small value means wealthy households, high values poorer households

Neg Correlations:
MOBI_REGIO: mobility patterns, where a higher value means low mobility, more stability
PLZ_ANTG1: share of small families
LP_STATUS_FEIN/GROB: Social status

So this component is positively correlated to share of bigger families (more densily populated, less wealthy) in the area and building and negatively correlated with social status and stability (younger, not wealthy tend to rent and change often) 

In [ ]:
# We set apart the most important positively and negatively correlated fetures
first_component_pos = ['PLZ8_ANTG3','KBA13_ANTG3','KBA13_ANTG4','KBA13_BAUMAX','PLZ8_ANTG4','PLZ8_BAUMAX','ANZ_HAUSHALTE_AKTIV','ANZ_STATISTISCHE_HAUSHALTE','CAMEO_DEUG_2015','WEALTH']
first_component_neg = ['LP_STATUS_GROB','MOBI_RASTER','LP_STATUS_FEIN','KBA05_ANTG1','KBA13_ANTG1','PLZ8_ANTG1','MOBI_REGIO']


### Second Component

In [ ]:
# Second component
pca_results(azdias_transformed, pca, 2)

Positive Corr:
KBA13_HERST_BMW_BENZ           0.1890: Share of BMW & Mercedes in the area
KBA13_SEG_OBEREMITTELKLASSE    0.1621: share of upper middle class cars and upper class cars (BMW5er, BMW7er etc.)
KBA13_MERCEDES                 0.1608: share of Mercedes in the area
KBA13_BMW : share of BMW in the area
KBA13_SEG_SPORTWAGEN: Share of sportcars
    
Negatice corr:
KBA13_SEG_KLEINWAGEN:    Share of small cars
KBA13_KMH_140_210 : Share of cars with max speed between 140 and 210
KBA13_HALTER_25, KBA13_HALTER_20: Young drivers

Looks like this component is positively correlated with uppr-middle class indicators like owning a BMW, Merc or Sportcar, mature drivers



In [ ]:
second_component_pos = ['KBA13_HERST_BMW_BENZ','KBA13_SEG_OBEREMITTELKLASSE','KBA13_MERCEDES','KBA13_BMW','KBA13_SITZE_4','KBA13_SEG_SPORTWAGEN']
second_component_neg = ['KBA13_HALTER_25','KBA13_KMH_180','KBA13_KMH_140_210','KBA13_SEG_KLEINWAGEN','KBA13_SITZE_5']

### Third Component

In [ ]:
pca_results(azdias_transformed, pca, 3)

Positively Corr:
KOMBIALTER                     0.2217
FINANZ_VORSORGER               0.1992: Financially: be preparer
CJT_TYP_5                      0.1927  Customer-Journey-Typology relating to the preferred information and buying channels of consumers
                                       Advertising- and Cross-Channel-Enthusiast
ALTERSKATEGORIE_GROB           0.1909 Age classification: higher -> older
CJT_TYP_4                      0.1834 advertisinginterested Online-shopper
CJT_TYP_3                      0.1782  advertisinginterested Store-shopper
CJT_TYP_6 Advertising-Enthusiast with restricted Cross-Channel-Behaviour 

Neg Corr
SEMIO_PFLICHT                 -0.1870 Traditionally minded (higher == less affinity)
FINANZ_UNAUFFAELLIGER         -0.1888 Financial Typology: unremarkable (higher = low)
FINANZ_ANLEGER                -0.1915 Investor? (high value = low likelihood)
CJT_TYP_2                     -0.2043 Advertising- and Consumptiontraditionalist
FINANZ_SPARER                 -0.2101 money saver (low value = very high)
CJT_TYP_1                     -0.2119 Advertising- and Consumptionminimalist
GENERATION (higher values == younger)

So this component is indicating the Customer Journey, Financial Attitude and Age. Higher component indicates higher probability of people being of higher age and money savers. The CJT_TYP_X variables indicate max affinity with the typology when value=1, lowest affinity when value=5


In [ ]:
third_component_pos = ['KOMBIALTER','FINANZ_VORSORGER','CJT_TYP_5','ALTERSKATEGORIE_GROB','CJT_TYP_4','CJT_TYP_3','CJT_TYP_6']
third_component_neg = ['SEMIO_PFLICHT','FINANZ_ANLEGER','FINANZ_UNAUFFAELLIGER','CJT_TYP_2','FINANZ_SPARER','CJT_TYP_1','GENERATION']


## Step 3: Clustering

### Step 3.1: Apply Clustering to General Population

In this step, you will apply k-means clustering to the dataset and use the average within-cluster distances from each point to their assigned cluster's centroid to decide on a number of clusters to keep.

- We use sklearn's [KMeans](http://scikit-learn.org/stable/modules/generated/sklearn.cluster.KMeans.html#sklearn.cluster.KMeans) class to perform k-means clustering on the PCA-transformed data.
- Then, we compute the average difference from each point to its assigned cluster's center.
- We perform the above two steps for a number of different cluster counts.
- We then select a final number of clusters to use, re-fit a KMeans instance to perform the clustering operation. We obtain the cluster assignments for the general demographics data.

I am using these functions instead of model.score to calculate the average distance of the points in the cluster 
from the centroid. I play with it first in two dimensions and then generalize to n-dimensions

Thanks to alphaleonis and Shay.G at https://stackoverflow.com/questions/40828929/sklearn-mean-distance-from-centroid-of-each-cluster for some of the code

uster import KMeans 

# Distance in two dimensions
def k_mean_distance(data, cx, cy, i_centroid, cluster_labels):
    # Calculate Euclidean distance for each data point assigned to centroid 
    distances = [np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[cluster_labels == i_centroid]]
    # return the mean value
    return np.mean(distances)


# Distance in n-dimensions
def k_mean_distance_n(data, centroid, i_centroid, cluster_labels):
    # Calculate Euclidean distance for each data point assigned to centroid 
    distances = [np.linalg.norm(x - centroid) for x in data[cluster_labels == i_centroid]]
    # return the mean value
    return np.mean(distances)

In [ ]:
from sklearn.cluster import MiniBatchKMeans, KMeans

# Distance in two dimensions
def k_mean_distance(data, cx, cy, i_centroid, cluster_labels):
    # Calculate Euclidean distance for each data point assigned to centroid 
    distances = [np.sqrt((x-cx)**2+(y-cy)**2) for (x, y) in data[cluster_labels == i_centroid]]
    # return the mean value
    return np.mean(distances)


# Distance in n-dimensions
def k_mean_distance_n(data, centroid, i_centroid, cluster_labels):
    # Calculate Euclidean distance for each data point assigned to centroid 
    distances = [np.linalg.norm(x - centroid) for x in data[cluster_labels == i_centroid]]
    # return the mean value
    return np.mean(distances)

In [ ]:
# We set the max num of clusters to 20
max_clusters = 30
# This will take a while
    
# compute the average within-cluster distances.
def k_means_score(data, n_clusters):
    # Run k-means
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=20000)
    #kmeans = KMeans(n_clusters=n_clusters)
    model = kmeans.fit(data)
    labels = model.predict(data)
    centroids = model.cluster_centers_
    
    total_distance = []
    for i, c in enumerate(centroids):
        # Function from above
        mean_distance = k_mean_distance_n(data, c, i, labels)
        total_distance.append(mean_distance)
    return(np.mean(total_distance))
  
scores = []
clusters = list(range(1,max_clusters+1))

import progressbar
cnter = 0
bar = progressbar.ProgressBar(maxval=len(clusters)+1, widgets=[progressbar.Bar('=', '[', ']'), ' ', progressbar.Percentage()])
bar.start()

for n_clusters in clusters:
    scores.append(k_means_score(azdias_pca, n_clusters))
    cnter+=1 
    bar.update(cnter)


bar.finish()    
print(scores)

In [ ]:
# We will use the elbow method to find the ideal number of clusters
plt.plot(clusters, scores, linestyle='--', marker='o', color='b');
plt.xlabel('No of Clusters');
plt.ylabel('Average Distance');
plt.title('Average Distance vs. Clusters');


Running MiniBatchKMeans instead of KMeans makes the graph a bit bumpier as the average distance does not always decreases with increasing n. of clusters. However, the general trend is the same. 
The graph above shows an elbow at 16 clusters and after 25 the average distance  keeps on decreasing at much lower pace.

In [ ]:
# Re-fit the k-means model with the selected number of clusters and obtain
# cluster predictions for the general population demographics data.
def apply_KMeans(df_pca, n_clusters):
    """
    
    """
    
    kmeans = MiniBatchKMeans(n_clusters=n_clusters, batch_size=50000)
    kmeans_model = kmeans.fit(df_pca)
    pop_labels = kmeans_model.predict(df_pca)
    pop_centroids = kmeans_model.cluster_centers_
    
    return pop_labels, pop_centroids, kmeans_model

gen_labels, gen_centroids, k_model = apply_KMeans(azdias_pca, 16)

In [ ]:
# Save to file in the current working directory
pkl_filename = "kmeans.pkl"  
with open(pkl_filename, 'wb') as file:  
    pickle.dump(k_model, file)

### Apply All Steps to the Customer Data

Now that we have clusters and cluster centers for the general population, we can see how the customer data maps on to those clusters. We will use the fits from the general population to clean, transform, and cluster the customer data. Finally we  will interpret how the general population fits apply to the customer data.

- Apply the same feature wrangling, selection, and engineering steps to the customer demographics using the `clean_data()` function you created earlier. (You can assume that the customer demographics data has similar meaning behind missing data patterns as the general demographics data.)
- Use the sklearn objects from the general demographics data, and apply their transformations to the customers data. That is, you should not be using a `.fit()` or `.fit_transform()` method to re-fit the old objects, nor should you be creating new sklearn objects! Carry the data through the feature scaling, PCA, and clustering steps, obtaining cluster assignments for all of the data in the customer demographics data.

In [ ]:
customers = pd.read_csv('arvato_data/Udacity_CUSTOMERS_052018.csv', sep=';')

In [ ]:
# Cleaning Data
#'CUSTOMER_GROUP', 'ONLINE_PURCHASE', and 'PRODUCT_GROUP columns will be dropped
customers = customers.drop(labels=['CUSTOMER_GROUP', 'ONLINE_PURCHASE', 'PRODUCT_GROUP'], axis=1)
customers = clean_data(customers, azdias_info)


In [ ]:
# Feature Transformation
#customers, imputers, scaler = feature_transformation(customers, azdias_info, imputers, scaler)
customers = c_transformer.transform(customers)

In [ ]:
# Applying Pca
cust_pca = pca.transform(customers)

In [ ]:
# Clustering (16 clusters)
customer_labels = k_model.predict(cust_pca)
customer_centroids = k_model.cluster_centers_

### Step 3.3: Compare Customer Data to Demographics Data

In [ ]:
np.bincount(gen_labels)/len(gen_labels)

In [ ]:
# Compare the proportion of data in each cluster for the customer data to the
# proportion of data in each cluster for the general population.

plt.figure(figsize=(10,5))

plt.subplot(1, 2, 1)
n_points = len(customer_labels)
max_count = np.max(np.bincount(customer_labels))
max_prop = max_count / n_points

# generate tick mark locations and names
tick_props = np.arange(0, max_prop, 0.05)
tick_names = ['{:0.2f}'.format(v) for v in tick_props]

# Customers the plot
base_color = sns.color_palette()[0]
sns.countplot(x = customer_labels, color = base_color)
plt.yticks(tick_props * n_points, tick_names)
plt.ylabel('proportion')
plt.xlabel('Cluster')
plt.title('Customers Clusters')
# Population plot
plt.subplot(1, 2, 2)
n_points = len(gen_labels)
max_count = np.max(np.bincount(gen_labels))
max_prop = max_count / n_points

sns.countplot(x = gen_labels, color = base_color)
plt.yticks(tick_props * n_points, tick_names)
plt.ylabel('proportion')
plt.xlabel('Cluster')
plt.title('General Population Clusters')
plt.tight_layout()



In [ ]:
# Better this ones
n_clusters=16
plt.figure(figsize=(10,5))
# These are the "Tableau 20" colors as RGB.    
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
  
# Scale the RGB values to the [0, 1] range, which is the format matplotlib accepts.    
for i in range(len(tableau20)):    
    r, g, b = tableau20[i]    
    tableau20[i] = (r / 255., g / 255., b / 255.) 

population_bins = 100 * np.bincount(gen_labels)/len(gen_labels)
customer_bins = 100 * np.bincount(customer_labels)/len(customer_labels)
clusters = [x for x in range(0,n_clusters)]
ax = plt.subplot(111)
w=0.4
ax.bar(x=np.array(clusters)-0.2, height=customer_bins, width=w, color=tableau20[0])
ax.bar(x=np.array(clusters)+0.2, height=population_bins, width=w, color=tableau20[2])
plt.ylabel('Proportion')
plt.xlabel('Cluster')
plt.xticks(clusters)
plt.legend(['Customers', 'General Population'])
plt.title('Customers vs Population Clusters')
plt.show()



We can see here that clusters 3, 7 and 14 are more represented in the customer dataset compared to the general population, while clusters 1, 2, 6, 10, 13, and 15 are under-represented.

From the previous analysis of the PCA we have:
- First Component: This component is positively correlated to indicators of wealth like WEALTH (lower values = wealthier), CAMEO_DEUG_2015 (lower values = upper class), ANZ_STATISTISCHE_HAUSHALTE and PLZ8_BAUMAX (# of households in the building, so higher values indicate more density and lower class). LP_STATUS_FEIN indicates higher income individual/families and MOBI_REGIO mobility (here we interpret is as higher mobility indicates less stability therefore lower income class) where higher values indicate lower mobility

- Second Component: This component is positively correlated with uppr-middle class indicators like owning a BMW, Merc or Sportcar, mature drivers. We will look at the variables: KBA13_HERST_BMW_BENZ, KBA13_SEG_OBEREMITTELKLASSE, KBA13_MERCEDES, KBA13_BMW, KBA13_SEG_SPORTWAGEN (sport ans luxury cars). And the negatively correlated KBA13_SEG_KLEINWAGEN, KBA13_KMH_140_210 (cheap cars) and KBA13_HALTER_25, KBA13_HALTER_20 (Young drivers)

- Third Component: This component is indicating the Customer Journey, positively correlated with younger and more 
multi-channel enthusiast people and negatively correlated with traditionalist, older people. We will look at the variables KOMBIALTER, FINANZ_VORSORGER, CJT_TYP_5, ALTERSKATEGORIE_GROB, CJT_TYP_4, CJT_TYP_3, CJT_TYP_6. 
Neg Correlated ones: SEMIO_PFLICHT, FINANZ_UNAUFFAELLIGER, FINANZ_ANLEGER, CJT_TYP_2, FINANZ_SPARER, CJT_TYP_1 and 
GENERATION (higher values == younger)

We will use the .inverse_transform() method of the PCA and StandardScaler objects to transform centroids back to the original data space and interpret the retrieved values directly.


In [ ]:
# We can have a look at the values of the centroids
customer_centroids[:, :3]

## To be reviewed:
So to start in Cluster 3 (4th row) we have a negative value for the first latent feature, suggesting a negative correlation as expected, since positive values in this feature point more towards low-income. The third feature at 3.93 point more towards younger, multi-channel and innvovative, and the 2nd feature moderatively towards higher income.
As for Cluster 7 (8th row), similar values for the 1st feature, strong positive for the second one (expensive cars) and moderatively positive for the third component.
Similarly in cluster 14 we have -5 in the 1st feature, but this time -2 in the second one (suggesting more younger population this time) and moderate pisitive for the customer journey (third component).
If we look at the clusters that are under-represented in the customer dataset, like cluster 2 with a strong first component indicating smaller, not stable households. Similarly cluster 6 has a high positive in the first component and high negative in teh second one (low-income + driving small cars). Cluster 10, 13 and 15 have all negative values in the third component, pointing towards older, traditionalist population in their customer-journey.
So from this first look it seems Customers are preferably richer, driving bigger cars but not old too established, prefering people with a more modern approach in their customer journey (multi-channel). 

Now we can try to tranform centroids back to their original data:

In [ ]:
# What kinds of people are part of a cluster that is overrepresented in the
# customer data compared to the general population?
#features_to_scale
centroids_df = pd.DataFrame(pca.inverse_transform(customer_centroids), columns=azdias_columns)
centroids_df[features_to_scale] = scaler.inverse_transform(centroids_df[features_to_scale])
#over_represented = [3, 7, 14]
#under_represented = [1, 2, 6, 10, 13, 15]
over_represented = [2, 6, 7, 10, 12]
under_represented = [3, 5, 11, 13, 14, 15]


In [ ]:
# First Component Customer Analysis
centroids_df[first_component_pos].iloc[over_represented].mean(), centroids_df[first_component_neg].iloc[over_represented].mean()

In [ ]:
centroids_df[first_component_pos].iloc[under_represented].mean(), centroids_df[first_component_neg].iloc[under_represented].mean()

If we look at WEALTH (lower values = wealthier) in the clusters that are more represented in the customers dataset the values are all around 2 indicating upper middleclass.
Instead in underrepresented clusters this value is sometimes higher than 4, pointing to lower classes.
Similarily CAMEO_DEUG_2015 (lower values = upper class), ANZ_STATISTISCHE_HAUSHALTE and PLZ8_BAUMAX (# of households in the building, so higher values indicate more density and lower class)
LP_STATUS_FEIN indicates higher income individual/families and MOBI_REGIO mobility (here we interpret is as higher mobility indicates less stability therefore lower income class) where higher values indicate lower mobility

In [ ]:
# Second component
centroids_df[second_component_pos].iloc[over_represented].mean(), centroids_df[second_component_neg].iloc[over_represented].mean()


In [ ]:
centroids_df[second_component_pos].iloc[under_represented].mean(), centroids_df[second_component_neg].iloc[under_represented].mean()

Here we can see that in the over represented clusters we have higher shares of luxury vehicles while in the under represented clusters we have higher shares of small, cheaper vehicles.

In [ ]:
# Third component
centroids_df[third_component_pos].iloc[over_represented].mean(), centroids_df[third_component_neg].iloc[over_represented].mean()

In [ ]:
centroids_df[third_component_pos].iloc[under_represented].mean(), centroids_df[third_component_neg].iloc[under_represented].mean()

Here we can see at the customer journey related variables CJT_TYP values of 4,5,6 indicate a more on-line shopper and cross channel enthusiast as opposed values of 1 and 2 indicate a more traditionalist approach. So as expected the more innovative type is in the under represented clusters while the more traditionalist in the over represented clusters.
Ther we have variables FINANZ_ indicating financial typology where SPARER, UNAUFFAELLIGER are more conservative while ANLEGER is more an investor. VORSORGER indicates financially prepared. GENERATION value is higher in the under represented indicating younger population.
More traditionalist, saving conscious and financially prepared are in the over represented cluster.

## Part 2: Supervised Learning Model

Now that you've found which parts of the population are more likely to be customers of the mail-order company, it's time to build a prediction model. Each of the rows in the "MAILOUT" data files represents an individual that was targeted for a mailout campaign. Ideally, we should be able to use the demographic information from each individual to decide whether or not it will be worth it to include that person in the campaign.

The "MAILOUT" data has been split into two approximately equal parts, each with almost 43 000 data rows. In this part, you can verify your model with the "TRAIN" partition, which i
ncludes a column, "RESPONSE", that states whether or not a person became a customer of the company following the campaign. In the next part, you'll need to create predictions on the "TEST" partition, where the "RESPONSE" column has been withheld.

In [2]:
#mailout_train = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')
mailout_train = pd.read_csv('arvato_data/Udacity_MAILOUT_052018_TRAIN.csv', sep=';')

/Users/maurizio/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [ ]:
mailout_train.shape

In [3]:
y = mailout_train.RESPONSE
X = mailout_train.drop('RESPONSE', axis=1)

In [ ]:
X = clean_data(X, azdias_info, cross_validation=True)
x_columns=X.columns
#X, imputers, scaler = feature_transformation(X, azdias_info)
X = c_transformer.transform(X)

In [4]:
# Save/Load in case the kernel dies
#pd.DataFrame(X, columns=x_columns).to_feather('X_mailout_train.feather')
X = pd.read_feather('X_mailout_train.feather', nthreads=2)

/Users/maurizio/anaconda3/lib/python3.6/site-packages/pandas/io/feather_format.py:112: FutureWarning: `nthreads` argument is deprecated, pass `use_threads` instead
  return feather.read_dataframe(path, nthreads=nthreads)


In [ ]:
# 
from sklearn.metrics import fbeta_score, accuracy_score, roc_auc_score

def train_predict(learner, sample_size, X_train, y_train, X_test, y_test): 
    '''
    inputs:
       - learner: the learning algorithm to be trained and predicted on
       - sample_size: the size of samples (number) to be drawn from training set
       - X_train: features training set
       - y_train: income training set
       - X_test: features testing set
       - y_test: income testing set
    '''
    
    results = {}
    
    # TODO: Fit the learner to the training data using slicing with 'sample_size' using .fit(training_features[:], training_labels[:])
    start = time() # Get start time
    learner.fit(X_train[:sample_size], y_train[:sample_size])
    end = time() # Get end time
    
    # TODO: Calculate the training time
    results['train_time'] = end - start
        
    # TODO: Get the predictions on the test set(X_test),
    #       then get predictions on the first 300 training samples(X_train) using .predict()
    start = time() # Get start time
    predictions_test = learner.predict(X_test)
    predictions_train = learner.predict(X_train)
    end = time() # Get end time
    
    # TODO: Calculate the total prediction time
    results['pred_time'] = end - start
            
    # TODO: Compute accuracy on the first 300 training samples which is y_train[:300]
    #results['acc_train'] = accuracy_score(y_train, predictions_train)
        
    # TODO: Compute accuracy on test set using accuracy_score()
    #results['acc_test'] = accuracy_score(y_test, predictions_test)
    
    # TODO: Compute F-score on the the first 300 training samples using fbeta_score()
    #results['f_train'] = fbeta_score(y_train, predictions_train, 0.5)
        
    # TODO: Compute F-score on the test set which is y_test
    #results['f_test'] = fbeta_score(y_test, predictions_test, 0.5)
    
    results['auc_test'] = roc_auc_score(y_test, predictions_test)
    results['auc_train'] = roc_auc_score(y_train, predictions_train)
        
    # Success
    print("{} trained on {} samples.".format(learner.__class__.__name__, sample_size))
        
    # Return the results
    return results

In [ ]:
# Import train_test_split
from sklearn.model_selection import train_test_split

# Split the 'features' and 'income' data into training and testing sets
X_train, X_CV, y_train, y_cv = train_test_split(X, y, test_size = 0.2, random_state = 0)

# Show the results of the split
print("Training set has {} samples.".format(X_train.shape[0]))
print("Testing set has {} samples.".format(X_CV.shape[0]))

In [ ]:
# TODO: Import the three supervised learning models from sklearn
from sklearn.model_selection import cross_val_score
from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression

# TODO: Initialize the three models
clf_A = LogisticRegression()

#clf_B = AdaBoostClassifier(random_state=0)
#clf_B = AdaBoostClassifier(base_estimator = DecisionTreeClassifier(criterion='gini', splitter='best', max_features=0.3,
#                                                                 max_depth=2, min_samples_split=0.7, min_samples_leaf=0.004,
#                                                                 min_weight_fraction_leaf=0., max_leaf_nodes=None, min_impurity_decrease=0., class_weight=None),
#                         random_state=0, learning_rate=0.45, n_estimators=4250)

clf_B = DecisionTreeClassifier(random_state=42)

clf_C = RandomForestClassifier(random_state=42)

clf_D = AdaBoostClassifier(random_state=42)


# TODO: Calculate the number of samples for 1%, 10%, and 100% of the training data
# HINT: samples_100 is the entire training set i.e. len(y_train)
# HINT: samples_10 is 10% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
# HINT: samples_1 is 1% of samples_100 (ensure to set the count of the values to be `int` and not `float`)
samples_100 = len(y_train)
samples_10 = len(y_train) // 10
samples_1 = len(y_train) // 100

# Collect results on the learners
results = {}
for clf in [clf_A, clf_B, clf_C, clf_D]:
    clf_name = clf.__class__.__name__
    results[clf_name] = {}
    samples = samples_100
    #for i, samples in enumerate([samples_1, samples_10, samples_100]):
    #results[clf_name] = train_predict(clf, samples, X_train, y_train, X_CV, y_cv)
    results[clf_name] = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')

# Run metrics visualization for the three supervised learning models chosen
#vs.evaluate(results, accuracy, fscore)
for clf in [clf_A, clf_B, clf_C, clf_D]:
    clf_name = clf.__class__.__name__
    #print(clf_name + " trained in {} seconds".format(results[clf_name]['train_time']))
    #print(clf_name + " Fbeta Training {}".format(results[clf_name]['f_train']))
    #print(clf_name + " Fbeta Test {}\n".format(results[clf_name]['f_test']))
    #print(clf_name + " ROC AUC Training {}\n".format(results[clf_name]['auc_train']))
    print(clf_name + " ROC AUC Test {}\n".format(results[clf_name]))
    

In [ ]:
from sklearn.model_selection import cross_val_score
#from sklearn.ensemble import AdaBoostClassifier, RandomForestClassifier, GradientBoostingClassifier
from xgboost import XGBClassifier
clf = XGBClassifier(random_state=42)
scores = cross_val_score(clf, X, y, cv=5, scoring='roc_auc')

scores

In [ ]:
# We will compare few methods by reusing the code from the "Finding Donors" project
# Learning Curves, to have a look at Bias/Variance
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier, GradientBoostingClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from plot_learning_curve import plot_learning_curve
from sklearn.model_selection import ShuffleSplit
from sklearn.metrics import make_scorer, roc_auc_score

#f_point5_scorer = make_scorer(fbeta_score, beta=0.5)

roc_scorer = make_scorer(roc_auc_score)

roc_scorer = 'roc_auc'
# TODO: Initialize the three models
clf_A = DecisionTreeClassifier(random_state=42)
clf_B = RandomForestClassifier(random_state=42)
clf_C = AdaBoostClassifier(random_state=42)
clf_D = GradientBoostingClassifier(random_state=42)

#X = pd.concat([X_train, X_CV])
#y = pd.concat([y_train,  y_cv])
plt.figure(figsize = [20, 5])
cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
plt.subplot(1, 4, 1)
plot_learning_curve(clf_A, 'Decision Tree Learning Curve', X, y, scorer=roc_scorer, cv=cv, ylim=(0.5, 1), n_jobs=2)

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
plt.subplot(1, 4, 2)
plot_learning_curve(clf_B, 'Random Forest Learning Curve', X, y, scorer=roc_scorer, cv=cv, ylim=(0.5, 1), n_jobs=2)

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
plt.subplot(1, 4, 3)
plot_learning_curve(clf_C, 'Ada Boost Learning Curve', X, y, scorer=roc_scorer, cv=cv, ylim=(0.5, 1), n_jobs=2)

cv = ShuffleSplit(n_splits=5, test_size=0.2, random_state=0)
plt.subplot(1, 4, 4)
plot_learning_curve(clf_D, 'Gradient Boosting Learning Curve', X, y, scorer=roc_scorer, cv=cv, ylim=(0.5, 1), n_jobs=2)

plt.show()

In [ ]:
xx = np.concatenate((X_train, X_CV))
xx.shape

## Part 3: Kaggle Competition

Now that you've created a model to predict which individuals are most likely to respond to a mailout campaign, it's time to test that model in competition through Kaggle. If you click on the link [here](http://www.kaggle.com/t/21e6d45d4c574c7fa2d868f0e8c83140), you'll be taken to the competition page where, if you have a Kaggle account, you can enter. If you're one of the top performers, you may have the chance to be contacted by a hiring manager from Arvato or Bertelsmann for an interview!

Your entry to the competition should be a CSV file with two columns. The first column should be a copy of "LNR", which acts as an ID number for each individual in the "TEST" partition. The second column, "RESPONSE", should be some measure of how likely each individual became a customer – this might not be a straightforward probability. As you should have found in Part 2, there is a large output class imbalance, where most individuals did not respond to the mailout. Thus, predicting individual classes and using accuracy does not seem to be an appropriate performance evaluation method. Instead, the competition will be using AUC to evaluate performance. The exact values of the "RESPONSE" column do not matter as much: only that the higher values try to capture as many of the actual customers as possible, early in the ROC curve sweep.

In [ ]:
mailout_test = pd.read_csv('../../data/Term2/capstone/arvato_data/Udacity_MAILOUT_052018_TEST.csv', sep=';')